In [1]:
import pandas as pd
from sklearn.utils import shuffle
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score

In [2]:
df1 = pd.read_excel('中國美國700筆標題.xlsx')
df2=df1.astype('str')
df2.index=list(range(len(df2)))

In [3]:
data = df2
dataset = data[['標籤', '新聞標題']]
dataset['label'] = dataset['標籤']
dataset['label']=dataset['label'].replace('2',0)
dataset['label']=dataset['label'].replace('1',1)
del dataset['標籤']
#dataset= shuffle(dataset)
dataset=pd.DataFrame(dataset,columns=['label','新聞標題'])
dataset.head()

C:\Anaconda\python\envs\GPU\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Anaconda\python\envs\GPU\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Anaconda\python\envs\GPU\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFram

,label,新聞標題
0,1,Tensions between Iran and Saudis could draw US...
1,1,"Trump offered to turn over fugitive, never tal..."
2,1,AT&T-Time Warner deal heading for big league l...
3,1,Thousands of Zimbabweans gather to march again...
4,1,"Examining Rangers extremes: Joe Thornton run, ..."


In [4]:
dataset['label']=dataset['label'].astype('int')

In [5]:
import multiprocessing

num_cores = multiprocessing.cpu_count()#計算可以使用多少內核來確定worker的數量
num_cores

12

In [6]:
#P.261 對每個文檔裡的每個word做分解，每個sample得到一個nx300的word2vector合集
#並合併每個word的word_vectors為sample_vecs(文檔的vector)，此時為每個文檔裡的所有word_vectors
#再把sample_vecs合併成vectorized_data，此時包含所有文檔的每個word的vector(頻率含意)
from nltk.tokenize import TreebankWordTokenizer

def tokenize_and_vectorize(dataset): 
    tokenizer = TreebankWordTokenizer() 
    vectorized_data = [] 
    expected = [] 
    for sample in dataset:#.iloc[:,1]: 
        tokens = tokenizer.tokenize(sample[1])#抓出sample的文字部分做分解
            
        vectorized_data.append(tokens) 
        
    return vectorized_data
token_list = tokenize_and_vectorize(dataset.values)
token_list

[['Tensions',
  'between',
  'Iran',
  'and',
  'Saudis',
  'could',
  'draw',
  'US',
  'into',
  'another',
  'war'],
 ['Trump',
  'offered',
  'to',
  'turn',
  'over',
  'fugitive',
  ',',
  'never',
  'talked',
  'human',
  'rights',
  ':',
  'Duterte',
  'spokesman'],
 ['AT',
  '&',
  'T-Time',
  'Warner',
  'deal',
  'heading',
  'for',
  'big',
  'league',
  'legal',
  'battle'],
 ['Thousands',
  'of',
  'Zimbabweans',
  'gather',
  'to',
  'march',
  'against',
  'Mugabe'],
 ['Examining',
  'Rangers',
  'extremes',
  ':',
  'Joe',
  'Thornton',
  'run',
  ',',
  'Ryan',
  'McDonagh',
  'sell-off'],
 ['North',
  'Korea',
  'media',
  'blasts',
  'Trump',
  'as',
  'an',
  '‘',
  'old',
  'lunatic',
  '’'],
 ['It',
  '’',
  's',
  'time',
  'for',
  'the',
  'Islanders',
  'to',
  'go',
  'all-in',
  'with',
  'a',
  'big',
  'trade'],
 ['Enes',
  'Kanter',
  'was',
  '‘',
  'crying',
  'like',
  'a',
  'baby',
  '’',
  'watching',
  'Knicks',
  '’',
  'ugly',
  'loss'],
 ['Enes

In [7]:
#產生一個token_list的Word2Vec
from gensim.models import Word2Vec

num_features = 300#word vector的向量元素的數量
min_word_count = 2#模型中要考慮的最小字數。語料庫很小，減少；語料庫很大，增加
num_workers = num_cores#train CPU的內核數
window_size = 6#上下文窗口大小
subsampling =1e-3#2-3#常用詞的二次採樣率

model = Word2Vec(token_list,
                 workers=num_workers, 
                 size=num_features, 
                 min_count=min_word_count, 
                 window=window_size, 
                 sample=subsampling)

In [8]:
from gensim.models import KeyedVectors

#word_vectors = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)
word_vectors =model

In [9]:
#P.261 對每個文檔裡的每個word做分解，每個sample得到一個nx300的word2vector合集
#並合併每個word的word_vectors為sample_vecs(文檔的vector)，此時為每個文檔裡的所有word_vectors
#再把sample_vecs合併成vectorized_data，此時包含所有文檔的每個word的vector(頻率含意)
from nltk.tokenize import TreebankWordTokenizer

def tokenize_and_vectorize(dataset): 
    tokenizer = TreebankWordTokenizer() 
    vectorized_data = [] 
    expected = [] 
    for sample in dataset:#.iloc[:,1]: 
        tokens = tokenizer.tokenize(sample[1])#抓出sample的文字部分做分解
        sample_vecs = [] 
        for token in tokens: 
            try: 
                sample_vecs.append(word_vectors[token])
                #word_vectors[token]每個字在word_vectors的vector(頻率含意) 
                
            except KeyError: 
                pass # No matching token in the Google w2v vocab 
            
        vectorized_data.append(sample_vecs) 
        
    return vectorized_data

In [10]:
vectorized_data = tokenize_and_vectorize(dataset.values)
expected = dataset.label

C:\Anaconda\python\envs\GPU\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  from ipykernel import kernelapp as app


In [11]:
pd.DataFrame(vectorized_data[0])

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,-0.000165,-0.007905,0.008403,0.003199,0.005325,0.001560,0.005349,0.005573,0.004205,0.005155,...,-0.007935,0.001257,-0.005444,0.004893,0.004584,-0.002290,0.003102,0.003232,0.005453,-0.005566
1,-0.001396,-0.013569,0.016219,0.003900,0.010236,0.002111,0.010116,0.010837,0.013082,0.010236,...,-0.015880,0.002197,-0.009137,0.005852,0.009140,-0.000460,0.008579,0.006998,0.007601,-0.007410
2,-0.005052,-0.080562,0.090683,0.024686,0.060142,0.012903,0.062731,0.057514,0.067134,0.060578,...,-0.096524,0.006787,-0.052368,0.039622,0.056709,-0.012688,0.054429,0.038398,0.053562,-0.050996
3,0.000170,-0.002379,0.005662,-0.000429,0.003366,0.000121,0.001997,0.003002,0.003509,0.002587,...,-0.004477,0.001556,-0.003240,0.000427,0.003824,-0.001789,0.001798,0.000513,0.003454,-0.001816
4,-0.003153,-0.043377,0.050164,0.012299,0.030801,0.007457,0.034592,0.029152,0.036318,0.033084,...,-0.053089,0.003553,-0.027529,0.020729,0.028656,-0.004576,0.027941,0.020016,0.028904,-0.026863
5,-0.001715,-0.004311,0.006415,0.002035,0.001689,0.001654,0.004984,0.002422,0.003220,0.004979,...,-0.005635,0.001560,-0.004271,0.003664,0.004219,-0.001880,0.004440,0.000958,0.002759,-0.003872
6,-0.007049,-0.099446,0.113280,0.029150,0.074683,0.015883,0.081516,0.072810,0.082868,0.076540,...,-0.121701,0.006857,-0.063040,0.048552,0.069862,-0.014142,0.068962,0.049189,0.066732,-0.061662
7,-0.000938,-0.016921,0.018816,0.004440,0.011631,0.003912,0.014965,0.013573,0.014386,0.011190,...,-0.018933,0.001517,-0.011447,0.008656,0.012157,-0.003827,0.010296,0.006664,0.009901,-0.010837
8,-0.001104,-0.011595,0.013957,0.002385,0.009405,0.003322,0.010367,0.009564,0.010706,0.006987,...,-0.013648,0.002332,-0.005886,0.005506,0.007279,-0.001024,0.007069,0.004923,0.006367,-0.008356
9,-0.006482,-0.084590,0.096404,0.025466,0.063681,0.013175,0.068493,0.061623,0.073184,0.064557,...,-0.104198,0.004036,-0.055696,0.043089,0.060469,-0.012401,0.059820,0.040943,0.056367,-0.051898


In [12]:
#切割資料
#split_point = int(len(vectorized_data)*.8)
#split_point_end = int(len(vectorized_data)*.5)

#x_train = vectorized_data[:split_point] 
x_train=vectorized_data[:630]+vectorized_data[700:1330]
y_train =pd.concat([expected[:630],expected[700:1330]])


#x_test = vectorized_data[split_point:-(int((len(vectorized_data)-split_point)*0.5))]#split_point_end]
#y_test = expected[split_point:-(int((len(vectorized_data)-split_point)*0.5))]#split_point_end]


In [13]:
test_T=vectorized_data[:705]
#test_T.index=range(len(test_T))
test_U=vectorized_data[630:700]
#test_U.index=range(len(test_U))
test_C=vectorized_data[1330:1400]
#test_C.index=range(len(test_C))

In [14]:
y_test_T=expected[:705]
#y_test_T.index=range(len(test_T))
y_test_U=expected[630:700]
#y_test_U.index=range(len(test_U))
y_test_C=expected[1330:1400]
#y_test_C.index=range(len(test_C))

In [15]:
#P.289 發現token長度設得太高，可調降長度至平均文檔token長度(Avg length)
def test_len(data, maxlen): 
    total_len = truncated = exact = padded = 0 
    for sample in data: 
        total_len += len(sample) 
        if len(sample) > maxlen: 
            truncated += 1 
        elif len(sample) < maxlen: 
            padded += 1 
        else: 
            exact +=1 
    print('Padded: {}'.format(padded)) 
    print('Equal: {}'.format(exact)) 
    print('Truncated: {}'.format(truncated)) 
    print('Avg length: {}'.format(total_len/len(data)))
    Avg_length = total_len/len(data)

In [16]:
test_len(vectorized_data, 600)

Padded: 1400
Equal: 0
Truncated: 0
Avg length: 9.247857142857143


In [17]:
#network的參數
maxlen = 400 #token的長度
batch_size = 32 #在反向傳播錯誤和更新權重之前要顯示多少個樣本
embedding_dims = 300 #token的word_vectors長度
epochs = 2 #整個訓練數據集通過網絡的次數

In [18]:
#P.262 padding(填充)不夠的data或truncate(截斷)多出的data
def pad_trunc(data, maxlen): 
    """ For a given dataset pad with zero vectors or truncate to maxlen """ 
    new_data = []
    # Create a vector of 0s the length of our word vectors 
    zero_vector = [] 
    for _ in range(len(data[0][0])):#每個文檔裡每個word的word_vectors(分數含意)長度都是300  
        zero_vector.append(0.0)#生成一個長度為300的zero vectors
        
    for sample in data:#每個文檔裡的字 
        if len(sample) > maxlen:#字的長度大於maxlen=500
            temp = sample[:maxlen] #只擷取到maxlen
        elif len(sample) < maxlen:
            temp = sample 
            # Append the appropriate number 0 vectors to the list 
            additional_elems = maxlen - len(sample) #不足500的長度
            for _ in range(additional_elems): 
                temp.append(zero_vector) #把不足的字的長度以zero vectors加上
        else: 
            temp = sample 
        new_data.append(temp) 
    return new_data

In [19]:
#P.262
import numpy as np

x_train = pad_trunc(x_train, maxlen)
test_U = pad_trunc(test_U, maxlen)
test_C = pad_trunc(test_C, maxlen)
test_T = pad_trunc(test_T, maxlen)
#轉為numpy array供Keras使用
#np.reshape(資料，(幾個文檔，token的長度，word_vector的長度))
x_train = np.reshape(x_train,(len(x_train), maxlen, embedding_dims))
y_train = np.array(y_train) 
test_U = np.reshape(test_U,(len(test_U), maxlen, embedding_dims))
y_test_U = np.array(y_test_U)
test_T = np.reshape(test_T,(len(test_T), maxlen, embedding_dims))
y_test_T= np.array(y_test_T)
test_C = np.reshape(test_C,(len(test_C), maxlen, embedding_dims))
y_test_C = np.array(y_test_C)



# RNN

In [20]:
#P.262 建模-使用遞歸神經網路(RNN)
#考量token的順序，一次放入一個token，試圖記住過去的output，看step t對step t+1的影響
#以此類推到文檔的最後一個token
from keras.models import Sequential 
from keras.layers import Dense, Dropout, Flatten, SimpleRNN

num_neurons = 50 #hidden neuron
model = Sequential()
model.add(SimpleRNN( 
    num_neurons, return_sequences=True,#return the network value at each time step
    input_shape=(maxlen, embedding_dims)))
model.add(Dropout(.2))#隨機關閉將傳到Dropout layer的輸入的百分比，減少學習特定的訓練集，免overfitting
model.add(Flatten()) #Dense layer需要一個由n個元素組成的“flat” vector作為input，因此使用Flatten()
model.add(Dense(1, activation='sigmoid'))#Dense layer，值在0(低於0.5)到1(高於0.5)之間#sigmoid
#model.add(Dense(1, activation='softmax'))
model.compile('rmsprop', 'binary_crossentropy', metrics=['accuracy'])
#因有打開或關閉的輸出神經元，所以使用binary_crossentropy
model.summary()#檢查模型的內部

Using TensorFlow backend.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_1 (SimpleRNN)     (None, 400, 50)           17550     
_________________________________________________________________
dropout_1 (Dropout)          (None, 400, 50)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 20000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 20001     
Total params: 37,551
Trainable params: 37,551
Non-trainable params: 0
_________________________________________________________________


In [25]:
#P.266 fit訓練模型
model.fit(x_train, y_train, 
           batch_size=batch_size, 
           epochs=epochs, 
           validation_data=(test_U, y_test_U)) 

Train on 1260 samples, validate on 70 samples
Epoch 1/2
1260/1260 [==============================] - 4s 3ms/step - loss: 0.6914 - accuracy: 0.5524 - val_loss: 0.8942 - val_accuracy: 0.2143
Epoch 2/2
1260/1260 [==============================] - 4s 3ms/step - loss: 0.7056 - accuracy: 0.5603 - val_loss: 0.5821 - val_accuracy: 0.7429


In [26]:
 from sklearn.metrics import classification_report

In [27]:
y_pred=model.predict_classes(test_C)
y_true=y_test_U
print(accuracy_score(y_true, y_pred))
print(metrics.precision_score(y_true, y_pred, average='binary'))
print(metrics.recall_score(y_true, y_pred, average='binary'))
print(metrics.f1_score(y_true, y_pred, average='binary') )
print(y_pred.sum())
print(y_true.sum())

0.6428571428571429
1.0
0.6428571428571429
0.782608695652174
45
70


In [28]:
y_pred=model.predict_classes(test_U)
y_true=y_test_U
print(accuracy_score(y_true, y_pred))
print(metrics.precision_score(y_true, y_pred, average='binary'))
print(metrics.recall_score(y_true, y_pred, average='binary'))
print(metrics.f1_score(y_true, y_pred, average='binary') )
print(y_pred.sum())
print(y_true.sum())

0.7428571428571429
1.0
0.7428571428571429
0.8524590163934427
52
70


In [ ]:
# y_pred=model.predict_classes(test_C)
# y_true=y_test_C
# print(accuracy_score(y_true, y_pred))
# print(metrics.precision_score(y_true, y_pred, average='binary'))
# print(metrics.recall_score(y_true, y_pred, average='binary'))
# print(metrics.f1_score(y_true, y_pred, average='binary') )



In [ ]:
y_pred=model.predict_classes(test_T)
a0=0
a1=0
for i in range(len(y_pred)):
    if y_pred[i][0] ==1:
        a0+=1
    else:
        a1+=1
print('a1=',a1)
print('a0=',a0)


# LSTM

In [ ]:
#P.286 建模
from keras.models import Sequential 
from keras.layers import Dense, Dropout, Flatten, LSTM 

num_neurons = 50 
model = Sequential() 
model.add(LSTM(num_neurons, return_sequences=True, input_shape=(maxlen, embedding_dims))) 
model.add(Dropout(.2)) 
model.add(Flatten()) 
model.add(Dense(1, activation='sigmoid')) 
#model.add(Dense(1, activation='softmax')) 
model.compile('rmsprop', 'binary_crossentropy', metrics=['accuracy']) 
model.summary() 

In [ ]:
#P.286 fit訓練模型
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(test_C, y_test_C)) 

In [ ]:
y_pred=model.predict_classes(test_C)
y_true=y_test_U
print(accuracy_score(y_true, y_pred))
print(metrics.precision_score(y_true, y_pred, average='binary'))
print(metrics.recall_score(y_true, y_pred, average='binary'))
print(metrics.f1_score(y_true, y_pred, average='binary') )

In [ ]:
y_pred=model.predict_classes(test_U)
y_true=y_test_U
print(accuracy_score(y_true, y_pred))
print(metrics.precision_score(y_true, y_pred, average='binary'))
print(metrics.recall_score(y_true, y_pred, average='binary'))
print(metrics.f1_score(y_true, y_pred, average='binary') )

In [ ]:
predict_y=model.predict_classes(test_U)
real_y=y_test_U
c=0
for i in range(len(predict_y)):
    if str(predict_y[i][0]) ==real_y[i]:
        c+=1
print(c/len(predict_y))    

In [ ]:
#P.287
vec_list = tokenize_and_vectorize((df_Tn[['標籤', '新聞標題']].astype('str')).values)

test_vec_list = pad_trunc(vec_list, maxlen)

test_vec = np.reshape(test_vec_list, (len(test_vec_list), maxlen, embedding_dims))

#print("Raw output of sigmoid function: {}".format(model.predict(test_vec))) 
model.predict_classes(test_vec)

In [ ]:
Tai=model.predict_classes(test_U)
a0=0
a1=0
for i in range(len(Tai)):
    if Tai[i][0] ==1:
        a0+=1
    else:
        a1+=1
print('a0=',a0)
print('a1=',a1)